In [1]:
with open('sudoku-easy.txt') as f:
    lines = f.readlines()

temp = [row.rstrip('\n') for row in lines]



n = 9
m = 9
puzzle = [0] * n
for i in range(n):
    puzzle[i] = [0] * m
    
for i in range(9):
    for j in range(9):
        puzzle[i][j] = int(temp[i][j])
        
puzzle

[[0, 0, 3, 0, 2, 0, 6, 0, 0],
 [0, 0, 0, 3, 0, 5, 0, 0, 1],
 [0, 0, 1, 8, 0, 6, 4, 0, 0],
 [0, 0, 8, 1, 0, 2, 9, 0, 0],
 [7, 0, 0, 0, 0, 0, 0, 0, 8],
 [0, 0, 6, 7, 0, 8, 2, 0, 0],
 [0, 0, 2, 6, 0, 9, 5, 0, 0],
 [8, 0, 0, 2, 0, 3, 0, 0, 9],
 [0, 0, 5, 0, 1, 0, 3, 0, 0]]

In [2]:
def zeroInd_recognizer(puzzle):
    for i in range(len(puzzle)):
        for j in range(len(puzzle[0])):
            if(puzzle[i][j] == 0):
                return i,j 
    return None

In [3]:
def forward_check( possibleVal, num, row, col):
    
    #check for future constraint in the same row
    for i in range(9):
        if i!=col:
            if(len(possibleVal[9*row + i]) == 1 and possibleVal[9*row + i][0] == num):
                return False
    
    #check for future constraint in the same col
    for i in range(9):
        if i!=row:
            if(len(possibleVal[9*i + col]) == 1 and possibleVal[9*i + col][0] == num):
                return False
            
            
     
    #check for future constraint in the same sub-Grid
    rowG = int(row / 3) * 3
    colG = int(col/3) * 3
    
    for i in range(rowG, rowG + 3):
        for j in range(colG, colG + 3):
            if(i!=row and j!=col):
                if(len(possibleVal[i*9 + j]) == 1 and possibleVal[i*9 + j][0] == num):
                    return False
            
                
    return True           

In [4]:
def update_possibleVal(possibleVal, row, col, num):
    
    #remove the value from all of the corresponding columns
    for i in range(9):
        if num in possibleVal[i+9*row]:
            possibleVal[i+9*row].remove(num)
            
    #remove the value from all of the corresponding rows
    for i in range(9):
        if num in possibleVal[col+9*i]:
            possibleVal[col+9*i].remove(num)
    
    
    #remove the value from all of the squares in the corresponding sub-Grid
    rowG = int(row/3)*3
    colG = int(col/3)*3  
    for i in range(rowG, rowG+3):
        for j in range(colG, colG+3):
            if num in possibleVal[j+ i*9]:
                possibleVal[j+ i*9].remove(num)

    return possibleVal

In [5]:
def createPossibleVal(puzzle):
    
    #create a 81*9 array, for for representing the possible values for each square
    possibleVal = [[] for i in range(81)]
    for i in range(81):
        for j in range(1,10):
            possibleVal[i].append(j)
    
    #find out all the squares that are already filled, 
    #then removing those numbers from possible options for othe squares
    #in their corresponding rows, columns, and sub-Grids
    for i in range(len(puzzle)):
        for j in range(len(puzzle[0])):
            if puzzle[i][j] != 0:
                
                #each square that is already filled, will have only [0] in the possibleVal array, 
                #so it won't create conflict later with others.
                possibleVal[i*9 + j] = [0] 
                num = puzzle[i][j]
                possibleVal = update_possibleVal(possibleVal, i, j, num)
            
    return possibleVal

In [6]:
def forwardChecking_solver(puzzle):
     
    #if there is no square with zero is found
    #means that the sudoku is solved
    if not zeroInd_recognizer(puzzle):
        return True
    
    #zeroInd_recognizer will return the cordinates of a square that has zero as its value
    #and needs to be set
    else:
        row, col = zeroInd_recognizer(puzzle)
    
    possibleVal = createPossibleVal(puzzle)
    l = possibleVal[row*9 + col]
    
    for i in l:
        
        #check for future constraint if choosing i value for corresponding row and col
        #if there is a square in the same row, or column or sub-Grid, that only has i as 
        #its option, then i will not be used in this row and col, and next i from l will 
        #be chosen to check next.
        if forward_check( possibleVal, i, row, col ):
            puzzle[row][col] = i
            print(puzzle)
            if forwardChecking_solver( puzzle ):
                return 1
            else:
                puzzle[row][col] = 0
            
    return False    

In [7]:
#function to print out the sudoku nicely
def print_p(puzzle):
    for i in range(len(puzzle)):
        for j in range(len(puzzle[0])):
            if j == 8:
                print(puzzle[i][j])
            else:
                print(str(puzzle[i][j]) + ",", end="")          

In [8]:
import time
    
start = time.time()
if(forwardChecking_solver(puzzle)):
    end = time.time()    
    print("backtracking-fc total time: ", end - start)
    print_p(puzzle)

else:
    print("No solution found")

[[4, 0, 3, 0, 2, 0, 6, 0, 0], [0, 0, 0, 3, 0, 5, 0, 0, 1], [0, 0, 1, 8, 0, 6, 4, 0, 0], [0, 0, 8, 1, 0, 2, 9, 0, 0], [7, 0, 0, 0, 0, 0, 0, 0, 8], [0, 0, 6, 7, 0, 8, 2, 0, 0], [0, 0, 2, 6, 0, 9, 5, 0, 0], [8, 0, 0, 2, 0, 3, 0, 0, 9], [0, 0, 5, 0, 1, 0, 3, 0, 0]]
[[4, 5, 3, 0, 2, 0, 6, 0, 0], [0, 0, 0, 3, 0, 5, 0, 0, 1], [0, 0, 1, 8, 0, 6, 4, 0, 0], [0, 0, 8, 1, 0, 2, 9, 0, 0], [7, 0, 0, 0, 0, 0, 0, 0, 8], [0, 0, 6, 7, 0, 8, 2, 0, 0], [0, 0, 2, 6, 0, 9, 5, 0, 0], [8, 0, 0, 2, 0, 3, 0, 0, 9], [0, 0, 5, 0, 1, 0, 3, 0, 0]]
[[4, 5, 3, 9, 2, 0, 6, 0, 0], [0, 0, 0, 3, 0, 5, 0, 0, 1], [0, 0, 1, 8, 0, 6, 4, 0, 0], [0, 0, 8, 1, 0, 2, 9, 0, 0], [7, 0, 0, 0, 0, 0, 0, 0, 8], [0, 0, 6, 7, 0, 8, 2, 0, 0], [0, 0, 2, 6, 0, 9, 5, 0, 0], [8, 0, 0, 2, 0, 3, 0, 0, 9], [0, 0, 5, 0, 1, 0, 3, 0, 0]]
[[4, 5, 3, 9, 2, 1, 6, 0, 0], [0, 0, 0, 3, 0, 5, 0, 0, 1], [0, 0, 1, 8, 0, 6, 4, 0, 0], [0, 0, 8, 1, 0, 2, 9, 0, 0], [7, 0, 0, 0, 0, 0, 0, 0, 8], [0, 0, 6, 7, 0, 8, 2, 0, 0], [0, 0, 2, 6, 0, 9, 5, 0, 0], [8, 0, 0, 